In [ ]:
import pandas as pd
import plotly.express as px
import pycountry

donnees = pd.read_csv('donnée.csv')
donnees = donnees[donnees['Year'] == donnees['Year'].max()]
donnees = donnees.rename(columns={
    'Entity': 'Pays',
    'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)': 'Consommation Alcool'
})

donnees = donnees.dropna(subset=['Consommation Alcool'])

def obtenir_code_iso(nom_pays):
    try:
        return pycountry.countries.lookup(nom_pays).alpha_3
    except:
        return None

donnees['Code ISO'] = donnees['Pays'].apply(obtenir_code_iso)
donnees = donnees.dropna(subset=['Code ISO'])

def ajouter_commentaire(consommation):
    if consommation > 15:
        return "🍻 Champion du monde !"
    elif consommation > 12:
        return "🥂 Très festif"
    elif consommation > 9:
        return "🍷 Amateur éclairé"
    elif consommation > 6:
        return "🍺 Modéré"
    elif consommation > 3:
        return "🍹 Occasionnel"
    else:
        return "🧃 Sobre ou abstinent"

donnees['Commentaire'] = donnees['Consommation Alcool'].apply(ajouter_commentaire)

carte = px.choropleth(
    donnees,
    locations='Code ISO',
    color='Consommation Alcool',
    hover_name='Pays',
    hover_data={
        'Consommation Alcool': ':.2f',
        'Commentaire': True,
        'Code ISO': False
    },
    color_continuous_scale='Reds',
    title='Consommation annuelle d\'alcool par habitant (litres d\'alcool pur)'
)

carte.update_layout(
    geo=dict(showframe=False, showcoastlines=False),
    coloraxis_colorbar=dict(title='Litres par an')
)

carte.show()

In [ ]:
continent_mapping = {
    'Europe': ['FRA', 'DEU', 'GBR', 'ITA', 'ESP', 'POL', 'ROU', 'NLD', 'BEL', 'GRC'],
    'Asie': ['CHN', 'JPN', 'KOR', 'IND', 'IDN', 'VNM', 'THA', 'MYS', 'PHL', 'SGP'],
    'Amérique': ['USA', 'CAN', 'MEX', 'BRA', 'ARG', 'COL', 'CHL', 'PER', 'VEN', 'URY'],
    'Afrique': ['ZAF', 'EGY', 'MAR', 'TUN', 'DZA', 'NGA', 'KEN', 'ETH', 'GHA', 'CIV'],
    'Océanie': ['AUS', 'NZL', 'PNG', 'FJI', 'NCL', 'VUT', 'SLB', 'WSM', 'TON']
}

def attribuer_continent(code_iso):
    for continent, pays in continent_mapping.items():
        if code_iso in pays:
            return continent
    return "Autre"

donnees['Continent'] = donnees['Code ISO'].apply(attribuer_continent)
moyennes_continent = donnees.groupby('Continent')['Consommation Alcool'].mean().sort_values(ascending=True)
fig_continents = px.bar(
    moyennes_continent,
    orientation='h',
    title='Consommation moyenne d\'alcool par continent',
    labels={'value': 'Litres d\'alcool par an', 'Continent': 'Continent'},
    color_discrete_sequence=['#ff6b6b']
)

fig_continents.update_layout(
    showlegend=False,
    xaxis_title="Litres d'alcool par an",
    yaxis_title="Continent"
)

fig_continents.show()